In [1]:
import numpy as np
import numpy.linalg as la

### Generate random rank-2 matrix

In [2]:
A = np.array ([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
               [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]).T

np.random.seed(0)
B = np.floor(np.random.rand (2, 18) * 10) 
X = A @ B 
print(f'X = \n {X}')
print("Shape of X", X.shape)
print(f'Rank X = {la.matrix_rank(X)}') 

X = 
 [[12. 15. 15. 12.  8. 13.  5. 14. 10. 12. 12.  9.  7. 16.  4.  5.  0. 14.]
 [17. 22. 21. 17. 12. 19.  9. 22. 19. 15. 19. 14. 12. 25.  4.  5.  0. 22.]
 [22. 29. 27. 22. 16. 25. 13. 30. 28. 18. 26. 19. 17. 34.  4.  5.  0. 30.]
 [27. 36. 33. 27. 20. 31. 17. 38. 37. 21. 33. 24. 22. 43.  4.  5.  0. 38.]
 [32. 43. 39. 32. 24. 37. 21. 46. 46. 24. 40. 29. 27. 52.  4.  5.  0. 46.]
 [37. 50. 45. 37. 28. 43. 25. 54. 55. 27. 47. 34. 32. 61.  4.  5.  0. 54.]
 [42. 57. 51. 42. 32. 49. 29. 62. 64. 30. 54. 39. 37. 70.  4.  5.  0. 62.]
 [47. 64. 57. 47. 36. 55. 33. 70. 73. 33. 61. 44. 42. 79.  4.  5.  0. 70.]
 [52. 71. 63. 52. 40. 61. 37. 78. 82. 36. 68. 49. 47. 88.  4.  5.  0. 78.]
 [57. 78. 69. 57. 44. 67. 41. 86. 91. 39. 75. 54. 52. 97.  4.  5.  0. 86.]]
Shape of X (10, 18)
Rank X = 2


### Add missing values

In [3]:
fractionObserved = .9 
np.random.seed(0)
Omega = np.array(np.random.rand(X.shape [0],X.shape [1]) < fractionObserved)
Xobs = Omega * X
# Where matrix = 0: we are missing
print(f'Observed X = \n {Xobs}')

Observed X = 
 [[12. 15. 15. 12.  8. 13.  5. 14.  0. 12. 12.  9.  7.  0.  4.  5.  0. 14.]
 [17. 22.  0. 17. 12. 19.  9. 22. 19.  0. 19. 14. 12. 25.  4.  5.  0. 22.]
 [22. 29.  0. 22. 16. 25. 13. 30. 28. 18. 26. 19. 17. 34.  4.  5.  0. 30.]
 [27. 36. 33. 27. 20. 31. 17. 38. 37. 21. 33. 24. 22. 43.  4.  5.  0. 38.]
 [ 0. 43. 39. 32. 24. 37. 21. 46. 46. 24. 40. 29. 27. 52.  4.  5.  0.  0.]
 [37. 50. 45. 37. 28. 43. 25. 54. 55. 27. 47. 34. 32.  0.  4.  5.  0. 54.]
 [42.  0. 51. 42. 32. 49. 29. 62. 64. 30. 54. 39. 37. 70.  0.  5.  0. 62.]
 [47. 64. 57. 47. 36. 55. 33. 70. 73. 33. 61. 44. 42. 79.  4.  5.  0. 70.]
 [52. 71. 63.  0. 40.  0. 37. 78. 82. 36. 68. 49. 47. 88.  4.  5.  0. 78.]
 [57. 78.  0. 57. 44. 67. 41. 86. 91. 39. 75. 54. 52. 97.  4.  0.  0. 86.]]


## Execute Truncated NNM algorithm

In [4]:
def shrinkage(X, tau):
    '''
    Return shrinkage D matrix from truncated NNM 
    '''
    U, S, V_T = la.svd(X, full_matrices=False)
    # shrink S
    S_star = np.where(S - tau < 0, 0, S - tau)
    S_hat = np.diag(S_star)
    return U @ S_hat @ V_T


In [5]:
def ADMM(A:np.array, B:np.array, X:np.array, params:dict) -> np.array:
    beta = params["beta"]
    # Initialize all variables as X
    X_k, W_k, Y_k = X, X, X

    for _ in range(params["max_iter_inner"]):
        # Update X
        X_k_1 = shrinkage(W_k - ((1/beta) * Y_k), tau=(1/beta))

        # update W
        W_k_1 = X_k_1 + (1/beta) * (A.T@B + Y_k)
        W_k_1 = (1 - Omega) * W_k_1 + Xobs 

        # update Y
        Y_k_1 = Y_k + beta * (X_k_1 - W_k_1)

        if la.norm(X_k_1 - X_k, ord='fro') < params["eps_inner"]:
            break

        # Update X, Y, W for next iteration
        X_k, Y_k, W_k = X_k_1, Y_k_1, W_k_1

    return X_k_1

In [6]:
# Truncated NNM
def truncated_NNM(rank:int, params:dict, orig_X:np.array) -> np.array:

    for _ in range(params["max_iter_outer"]):
        
        # Take SVD of X_observed
        U, S, V_T = la.svd(orig_X , full_matrices=True)
        V = V_T.T
        # Get truncated U and V as A and B
        A = U[:, :rank].T
        B = V[:, :rank].T

        # Perform ADMM minimization
        new_X = ADMM(A, B, orig_X, params)

        if la.norm(new_X - orig_X, ord='fro') < params["eps_outer"]:
            break
        
        # Else, update X
        orig_X = new_X
    np.set_printoptions(suppress=True, precision=2)
    print(f"FINAL X = \n {new_X}")
    return new_X

### Test algorithm

In [7]:
parameters = {"eps_outer": .001,
              "eps_inner": .001,
              "beta": 1,
              "max_iter_outer": 1000,
              "max_iter_inner": 1000}

new_X = truncated_NNM(rank=2, 
                      params=parameters, 
                      orig_X=Xobs)

FINAL X = 
 [[12. 15. 15. 12.  8. 13.  5. 14. 10. 12. 12.  9.  7. 16.  4.  5.  0. 14.]
 [17. 22. 21. 17. 12. 19.  9. 22. 19. 15. 19. 14. 12. 25.  4.  5.  0. 22.]
 [22. 29. 27. 22. 16. 25. 13. 30. 28. 18. 26. 19. 17. 34.  4.  5.  0. 30.]
 [27. 36. 33. 27. 20. 31. 17. 38. 37. 21. 33. 24. 22. 43.  4.  5.  0. 38.]
 [32. 43. 39. 32. 24. 37. 21. 46. 46. 24. 40. 29. 27. 52.  4.  5.  0. 46.]
 [37. 50. 45. 37. 28. 43. 25. 54. 55. 27. 47. 34. 32. 61.  4.  5.  0. 54.]
 [42. 57. 51. 42. 32. 49. 29. 62. 64. 30. 54. 39. 37. 70.  4.  5.  0. 62.]
 [47. 64. 57. 47. 36. 55. 33. 70. 73. 33. 61. 44. 42. 79.  4.  5.  0. 70.]
 [52. 71. 63. 52. 40. 61. 37. 78. 82. 36. 68. 49. 47. 88.  4.  5.  0. 78.]
 [57. 78. 69. 57. 44. 67. 41. 86. 91. 39. 75. 54. 52. 97.  4.  5.  0. 86.]]
